In [5]:
!pip install -q jupysql sqlalchemy pymysql
%load_ext sql

from getpass import getpass
import urllib.parse

dbUser = "root"
dbHost = "127.0.0.1"
dbPort = 3306
dbName = "wonderlandElections"

dbPass = getpass("MySQL password: ")
encodedPass = urllib.parse.quote_plus(dbPass)

conn_url = f"mysql+pymysql://{dbUser}:{encodedPass}@{dbHost}:{dbHost}/{dbName}"

MySQL password:  ········


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%config SqlMagic.displaylimit = None
%config SqlMagic.named_parameters="enabled"

In [ ]:
# USER GUIDE
# ENTER CORRECT VALUES WHEN PROMPTED

In [ ]:
#A.1
#CLERK CREATES A NEW POLL (ENTER IN A VALID CLERK)
from datetime import datetime

while True:
    clerkID = input("Enter your Clerk Folk ID: ")
    clerkExists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM PollStaff WHERE folkID='{clerkID}' AND job='clerk' LIMIT 1;"
    )
    if clerkExists:
        print(f"Clerk verified: {clerkID}")
        break
    else:
        print(f"Folk {clerkID} not authorized")

while True:
    pollID = input("Enter new Poll ID (4 varchars): ")
    if len(pollID) != 4:
        print("[ERROR] Poll ID muyst be exactly 4 varchars.")
        continue
    break
    
question = input("Enter poll question text: ")

def checkDatetime(prompt):
    while True:
        dt = input(prompt)
        try:
            return datetime.strptime(dt,  "%Y-%m-%d %H:%M:%S")
        except ValueError:
            print("[ERROR] Use format YYYY-MM-DD HH:MM:SS")

while True:
    startTime = checkDatetime("Enter poll start (YYYY-MM-DD HH:MM:SS): ")
    endTime = checkDatetime("Enter poll end (YYYY-MM-DD HH:MM:SS): ")
    
    if(endTime > startTime):
        break
    print("[ERROR] Poll end should be after Poll start")

In [ ]:
try:
    get_ipython().run_cell_magic("sql", "", "START TRANSACTION;")

    exists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM Poll WHERE pollID='{pollID}' LIMIT 1;"
    )
    if not exists:
        get_ipython().run_cell_magic("sql", "",
            f""" INSERT INTO Poll (pollID, questionText, periodStart, periodEnd)
            VALUES ('{pollID}', '{question}', '{startTime}', '{endTime}');"""
        )
        get_ipython().run_cell_magic("sql", "", "COMMIT;")
        print(f"[CLERK] Poll '{pollID}' created successfully.")
    else:
        get_ipython().run_cell_magic("sql", "", "ROLLBACK;")
        print(f"[CLERK] Poll '{pollID}' already exists.")

except Exception as e:
    get_ipython().run_cell_magic("sql", "", "ROLLBACK;")
    print("[CLERK] Failed to create poll.")
    raise

In [ ]:
#A.2
# FOLK CREATES REGISTRATION
# ENTER A FOLK WHO EXISTS
# FOLK -> 16 NUMBERS
# REGISTRATION ID -> INT
# VOTING CENTER ID -> 4 ALPHANUMERIC CHARACTERS
# POLL ID -> 4 ALPHANUMERIC CHARACTERS
while True:
    registrationID = input("Enter a Registration ID: ")
    if registrationID.isdigit():
        registrationID = int(registrationID)
        break
    print("[ERROR] Registration ID must be numeric.")

while True:
    folkID = input("Enter a Folk ID: ")
    exists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM folk WHERE folkID='{folkID}' LIMIT 1;"
    )
    if not exists:
        print("[ERROR] Folk does not exist.")
        continue
    break
while True:
    votingCenterID = input("Enter a Voting Center ID: ")

    if len(votingCenterID) != 4:
        print("[ERROR] Voting Center ID must be exactly 4 varchars.")
        continue

    exists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM Location WHERE votingCenterID='{votingCenterID}'LIMIT 1;"
    )
    if exists:
        break
    else:
        print("[ERROR] Voting Center does not exist.")

while True:
    pollID = input("Enter Poll ID (4 varchars): ")
    if len(pollID) != 4:
        print("[ERROR] Poll ID muyst be exactly 4 varchars.")
        continue
    exists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM Poll WHERE pollID='{pollID}'LIMIT 1;"
    )
    if exists:
        break
    else:
        print("[ERROR] Poll does not exist.")

def checkDatetime(prompt):
    while True:
        dt = input(prompt)
        try:
            return datetime.strptime(dt,  "%Y-%m-%d %H:%M:%S")
        except ValueError:
            print("[ERROR] Use format YYYY-MM-DD HH:MM:SS")

voteDate = checkDatetime("Enter a voting date: ")

In [ ]:
try:
    get_ipython().run_cell_magic("sql", "", "START TRANSACTION;")

    regExists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM registration WHERE registrationID='{registrationID}' LIMIT 1;"
    )
    
    if not exists:
        get_ipython().run_cell_magic("sql", "",
            f""" INSERT INTO registration (registrationID, folkID, votingCenterID, pollID, voteDate)
            VALUES ('{registrationID}', '{folkID}', '{votingCenterID}', '{pollID}', '{voteDate}');"""
        )
        get_ipython().run_cell_magic("sql", "", "COMMIT;")
        print(f"[FOLK] Registration '{registrationID}' created successfully.")
    else:
        get_ipython().run_cell_magic("sql", "", "ROLLBACK;")
        print(f"[FOLK] Registration '{registrationID}' already exists.")

except Exception as e:
    get_ipython().run_cell_magic("sql", "", "ROLLBACK;")
    print("[FOLK] Failed to register.")
    raise

In [ ]:
#A.3
# CLERK CHANGES POLL TIMES (ENTER IN A VALID CLERK)
# POLL ID -> 4 ALPHANUMERIC CHARACTERS

from datetime import datetime

def checkDatetime(prompt):
    while True:
        dt = input(prompt)
        try:
            return datetime.strptime(dt,  "%Y-%m-%d %H:%M:%S")
        except ValueError:
            print("[ERROR] Use format YYYY-MM-DD HH:MM:SS")

while True:
    clerkID = input("Enter your Clerk Folk ID: ")
    clerkExists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM PollStaff WHERE folkID='{clerkID}' AND job='clerk' LIMIT 1;"
    )
    if clerkExists:
        print(f"Clerk verified: {clerkID}")
        break
    else:
        print(f"Folk {clerkID} not authorized")

while True:
    pollID = input("Enter new Poll ID (4 varchars): ")
    if len(pollID) != 4:
        print("[ERROR] Poll ID muyst be exactly 4 varchars.")
        continue
    exists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM Poll WHERE pollID='{pollID}'LIMIT 1;"
    )
    if exists:
        break
    else:
        print("[ERROR] Poll does not exist.")

while True:
    newStart = checkDatetime("Enter NEW poll start (YYYY-MM-DD HH:MM:SS): ")
    newEnd = checkDatetime("Enter NEW poll end   (YYYY-MM-DD HH:MM:SS): ")

    if newEnd > newStart:
        break
    print("[ERROR] Poll end must be after poll start.")

In [ ]:
try:
    get_ipython().run_cell_magic("sql", "", "START TRANSACTION;")
    get_ipython().run_cell_magic(
        "sql", "", f"""
            UPDATE Poll
            SET periodStart = '{newStart:%Y-%m-%d %H:%M:%S}',
                periodEnd   = '{newEnd:%Y-%m-%d %H:%M:%S}'
            WHERE pollID = '{pollID}';
            """
        )
    get_ipython().run_cell_magic("sql", "", "COMMIT;")
    print(f"[CLERK] Poll '{pollID}' availability updated successfully.")
except Exception:
    get_ipython().run_cell_magic("sql", "", "ROLLBACK;")
    print("[CLERK] Failed to modify poll availability (registrations or ballots may exist).")
    raise

In [ ]:
#A.4
# VOTER CASTS BALLOT 
# FOLK ID -> 16 NUMBERS
# REGISTRATION ID -> INT

from datetime import datetime

def checkDatetime(prompt):
    while True:
        dt = input(prompt)
        try:
            return datetime.strptime(dt,  "%Y-%m-%d %H:%M:%S")
        except ValueError:
            print("[ERROR] Use format YYYY-MM-DD HH:MM:SS")

while True:
    folkID = input("Enter your Folk ID: ")
    folkExists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM Folk WHERE folkID='{folkID}' LIMIT 1;"
    )
    if folkExists:
        break
    else:
        print("Folk does not exist")

voteDate = checkDatetime("Enter a voting date: ")

while True:
    vote = input("Enter vote [yes, no, abstain]: ")

    if vote != 'yes' and vote != 'no' and vote != 'abstain':
        print("[ERROR] Vote must be 'yes', 'no', or 'abstain'.")
    else:
        break

while True:
    registrationID = input("Enter a Registration ID: ")
    if registrationID.isdigit():
        registrationID = int(registrationID)
        break
    print("[ERROR] Registration ID must be numeric.")

try:
    get_ipython().run_cell_magic("sql", "", f""" call castBallot({registrationID}, '{voteDate:%Y-%m-%d %H:%M:%S}', '{vote}', '{folkID}');""")
    print("[VOTER] Ballot cast successfully")
except Exception as e:
    print("[VOTER] Ballot cast failed.")
    print("Details: ", e)

In [ ]:
#A.5
# REMOVES A FOLK AND ALL ASSOCIATED INFORMATION
# ENTER IN A POLL STAFF (FOLK ID)
# FOLK ID -> 16 NUMBERS
while True:
    pollStaffID = input("Enter your Poll Staff ID: ")
    pollStaffExists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM PollStaff WHERE folkID='{pollStaffID}' LIMIT 1;"
    )
    if pollStaffExists:
        print(f"Clerk verified: {pollStaffID}")
        break
    else:
        print(f"Folk {pollStaffID} not authorized")

while True:
    folkID = input("Enter Folk ID: ")
    folkExists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM Folk WHERE folkID='{folkID}' LIMIT 1;"
    )
    if folkExists:
        break
    else:
        print("Folk does not exist")

In [ ]:
try:
    get_ipython().run_cell_magic("sql", "", "START TRANSACTION;")
    get_ipython().run_cell_magic("sql", "", f""" delete b 
                                                from Ballot b 
                                                join Registration r 
                                                on r.registrationID = b.registrationID
                                                where r.folkID='{folkID}';""")
    get_ipython().run_cell_magic("sql", "", f" delete from Registration where folkID='{folkID}';")
    get_ipython().run_cell_magic("sql", "", f" delete from PollStaffShift where folkID='{folkID}';")
    get_ipython().run_cell_magic("sql", "", f" delete from PollStaff where folkID='{folkID}';")
    get_ipython().run_cell_magic("sql", "", f" delete from FolkEmail where folkID='{folkID}';")
    get_ipython().run_cell_magic("sql", "", f" delete from Folk where folkID='{folkID}';")
    get_ipython().run_cell_magic("sql", "", "COMMIT;")
    print(f"[STAFF] Folk '{folkID}' successfully deleted.")

except Exception as e:
    get_ipython().run_cell_magic("sql", "", "ROLLBACK;")
    print("[STAFF] Failed to delete folk.")
    raise

In [ ]:
#B.1
# LISTS NAME, CITY, AND EMAIL OF ALL FOLKS

%%sql
select f.firstName, f.lastName, l.city, MIN(fe.email) AS email
from Folk f
join Location l
  on f.xCoord = l.xCoord and f.yCoord = l.yCoord
left join FolkEmail fe
  on fe.folkID = f.folkID
group by f.folkID, f.firstName, f.lastName, l.city
order by l.city, f.lastName, f.firstName;

In [ ]:
#B.2
# LIST THE CITY, STATE, AND POPULATION OF EACH CITY IN DECREASING ORDER

%%sql
select l.city, l.state,
    count(*) as num_residents
from Folk f
join Location l
on f.xCoord = l.xCoord and f.yCoord = l.yCoord
group by l.city, l.state
order by num_residents desc;

In [ ]:
#B.3
# LIST EACH FOLK IN EACH CENTER IN INCREASING ALPHABETICAL ORDER OF THEIR ZIPCODE

%%sql
select l.votingCenterID as center, l.zipCode, l.city, l.state,
    count(distinct r.folkID) as numCurrentlyRegisteredFolks
from Location l
left join Registration r
on r.votingCenterID = l.votingCenterID
where l.locationType = 'votingCenter'
group by l.votingCenterID, l.zipCode, l.city, l.state
order by l.zipCode ASC, l.votingCenterID ASC;

In [ ]:
#B.4
# FINDS ALL DISTINCT FOLKS WHO REGISTERED AT A GIVEN VOTING CENTER
# VOTING CENTER ID -> 4 ALPHANUMERIC CHARACTERS

from datetime import datetime

def checkDatetime(prompt):
    while True:
        dt = input(prompt)
        try:
            return datetime.strptime(dt,  "%Y-%m-%d %H:%M:%S")
        except ValueError:
            print("[ERROR] Use format YYYY-MM-DD HH:MM:SS")


while True:
    votingCenterID = input("Enter a Voting Center ID: ")

    if len(votingCenterID) != 4:
        print("[ERROR] Voting Center ID must be exactly 4 varchars.")
        continue

    exists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM Location WHERE votingCenterID='{votingCenterID}'LIMIT 1;"
    )
    if exists:
        break
    else:
        print("[ERROR] Voting Center does not exist.")

while True:
    startDT = checkDatetime("Enter start datetime (YYYY-MM-DD HH:MM:SS): ")
    endDT = checkDatetime("Enter end datetime   (YYYY-MM-DD HH:MM:SS): ")

    if newEnd > newStart:
        break
    print("[ERROR] End datetime must be after start datetime.")

In [ ]:
%%sql
select distinct f.folkID, f.firstName, f.lastName
from Registration r
join Folk f
on f.folkID = r.folkID
where r.votingCenterID = :centerID and r.voteDate between :startDT and :endDT
order by f.lastName, f.firstName, f.folkID;

In [ ]:
#B.5
# GIVES THE NUMBER OF UNIQUE REGISTRATIONS FOR ANY VOTING CENTER WITHIN 3 MILES OF 0, 0 EXCEPT FOR THOSE EXCLUDED
# VOTING CENTER ID -> 4 ALPHANUMERIC CHARACTERS
# ENTER continue WHEN DONE WITH ADDING VOTING CENTERS TO BE EXCLUDED

from datetime import datetime

def checkDatetime(prompt):
    while True:
        dt = input(prompt)
        try:
            return datetime.strptime(dt,  "%Y-%m-%d %H:%M:%S")
        except ValueError:
            print("[ERROR] Use format YYYY-MM-DD HH:MM:SS")
            
exclusionList = []

while True:
    votingCenterID = input("Enter a Voting Center ID [enter continue to break loop]: ")

    if votingCenterID == 'continue':
        break
        
    if len(votingCenterID) != 4:
        print("[ERROR] Voting Center ID must be exactly 4 varchars.")
        continue

    exists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM Location WHERE votingCenterID='{votingCenterID}'LIMIT 1;"
    )
    if exists:
        exclusionList.append(votingCenterID)
    else:
        print("[ERROR] Voting Center does not exist.")
if exclusionList:
    exclusionSql = "(" + ",".join(f"'{c}'" for c in exclusionList) + ")"
else:
    exclusionSql = ""

while True:
    startDT = checkDatetime("Enter start datetime (YYYY-MM-DD HH:MM:SS): ")
    endDT = checkDatetime("Enter end datetime   (YYYY-MM-DD HH:MM:SS): ")

    if newEnd > newStart:
        break
    print("[ERROR] End datetime must be after start datetime.")

In [ ]:
%%sql
select
    count(distinct r.registrationID) as unique_registrations
from Registration r
join Location l
on l.votingCenterID = r.votingCenterID
where l.locationType = 'votingCenter'
and calculateDistance(l.xCoord, l.yCoord) <= 3
{{exclusionSql}}
and r.voteDate >= :startDT
and r.voteDate < :endDT;

In [ ]:
#B.6
# FINDS THE MOST REGISTERED TO VOTING CENTERS IN A GIVEN CITY IN A GIVEN TIME FRAM
# CITIES -> MEGAPOLIS, CASTELIA, WILMINGTON

from datetime import datetime

def checkDatetime(prompt):
    while True:
        dt = input(prompt)
        try:
            return datetime.strptime(dt,  "%Y-%m-%d %H:%M:%S")
        except ValueError:
            print("[ERROR] Use format YYYY-MM-DD HH:MM:SS")

while True:
    startDT = checkDatetime("Enter start datetime (YYYY-MM-DD HH:MM:SS): ")
    endDT = checkDatetime("Enter end datetime   (YYYY-MM-DD HH:MM:SS): ")

    if newEnd > newStart:
        break
    print("[ERROR] End datetime must be after start datetime.")

while True:
    city = input("Enter a City: ")

    cityExists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM Location WHERE city='{city}'LIMIT 1;"
    )
    if cityExists:
        break
    else:
        print("[ERROR] City does not exist.")

In [ ]:
%%sql
with centerCounts as (
  select
      l.votingCenterID,
      count(*) as totalRegs
  from Location l
  join Registration r
    on r.votingCenterID = l.votingCenterID
  where l.locationType = 'votingCenter' and l.city = :city and r.voteDate between :startDT and :endDT
  group by l.votingCenterID
)
select votingCenterID, totalRegs
from centerCounts
where totalRegs = (select MAX(totalRegs) from centerCounts)
order by votingCenterID;

In [ ]:
#B.7
# FINDS ALL FOLK WHO HAVE REGISTERED TO EVERY VOTING CENTER IN THEIR STATE
# STATES -> CAPITAL DISTRICT, UNOVA, DELAWARE

while True:
    state = input("Enter a State: ")

    stateExists = get_ipython().run_cell_magic(
        "sql", "", f"SELECT 1 FROM Location WHERE state='{state}'LIMIT 1;"
    )
    if stateExists:
        break
    else:
        print("[ERROR] State does not exist.")

In [ ]:
%%sql
select
    f.folkID, f.firstName, f.lastName
from Folk f
join Registration r
on r.folkID = f.folkID
join Location l
on l.votingCenterID = r.votingCenterID
where l.locationType = 'votingCenter' and l.state = :state
group by f.folkID, f.firstName, f.lastName
having count(distinct r.votingCenterID) = (
    select count(*)
    from Location nxtL
    where nxtL.locationType = 'votingCenter' and nxtL.state = :state
)
ORDER BY f.lastName, f.firstName, f.folkID;

In [ ]:
#B.8
# FINDS ALL FOLK WHO REGISTERED TO A VOTING CENTER NOT CLOSEST TO THEIR HOME

%%sql
select distinct 
    f.folkID, f.firstName, f.lastName
from Folk f
join Registration r
on r.folkID = f.folkID
join Location regC
on regC.votingCenterID = r.votingCenterID and regC.locationType = 'votingCenter'
where
  distanceBetweenPoints(f.xCoord, f.yCoord, regC.xCoord, regC.yCoord) > (
    select distanceBetweenPoints(f.xCoord, f.yCoord, vc.xCoord, vc.yCoord)
    from Location vc
    where vc.locationType = 'votingCenter'
    order by
      distanceBetweenPoints(f.xCoord, f.yCoord, vc.xCoord, vc.yCoord) ASC,
      vc.votingCenterID asc
    limit 1
  )
order by f.lastName, f.firstName, f.folkID;